In [1]:
import os
import openai
import sys
sys.path.append('../..')

import constants
openai.api_key = constants.OPENAI_API_KEY

In [3]:
from langchain.vectorstores import Chroma
from langchain.embeddings.openai import OpenAIEmbeddings
persist_directory = 'docs/chroma/'
embedding = OpenAIEmbeddings(openai_api_key=constants.OPENAI_API_KEY)
vectordb = Chroma(persist_directory=persist_directory, embedding_function=embedding)

In [4]:
print(vectordb._collection.count())

104


In [5]:
question = "Wie arbeiten wir mit TIS?"
docs = vectordb.similarity_search(question,k=3)

In [6]:
len(docs)

3

In [8]:
from langchain.chat_models import ChatOpenAI
llm = ChatOpenAI(openai_api_key=constants.OPENAI_API_KEY, model_name="gpt-3.5-turbo", temperature=0)

In [9]:
from langchain.chains import RetrievalQA

In [ ]:
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever = vectordb.as_retriever()
)